In [ ]:
# docker run -it \
#     --rm \
#     --name elasticsearch \
#     -p 9200:9200 \
#     -p 9300:9300 \
#     -e "discovery.type=single-node" \
#     -e "xpack.security.enabled=false" \
#     docker.elastic.co/elasticsearch/elasticsearch:8.4.3

In [1]:
import requests 

docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [2]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [3]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course-questions-hw1"


from elasticsearch import Elasticsearch

es_client = Elasticsearch('http://localhost:9200')
es_client.indices.create(index=index_name, body = index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions-hw1'})

In [40]:
!curl localhost:9200
# Q1

{
  "name" : "cc59a4f0f285",
  "cluster_name" : "docker-cluster",
  "cluster_uuid" : "rwYg-pt7QmaEmyb1qCVsXA",
  "version" : {
    "number" : "8.4.3",
    "build_flavor" : "default",
    "build_type" : "docker",
    "build_hash" : "42f05b9372a9a4a470db3b52817899b99a76ee73",
    "build_date" : "2022-10-04T07:17:24.662462378Z",
    "build_snapshot" : false,
    "lucene_version" : "9.3.0",
    "minimum_wire_compatibility_version" : "7.17.0",
    "minimum_index_compatibility_version" : "7.0.0"
  },
  "tagline" : "You Know, for Search"
}


In [5]:
from tqdm.auto import tqdm

/usr/local/python/3.10.13/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
#Q2
for doc in tqdm(documents):
    es_client.index(index = index_name, document = doc)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 948/948 [00:24<00:00, 38.58it/s]


In [7]:
es_client.count(index=index_name)

ObjectApiResponse({'count': 948, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

In [35]:
#Q3
query = 'How do I execute a command in a running docker container?'
serch_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
            # ,
            # "filter": {
            #     "term": {
            #         "course": "machine-learning-zoomcamp"
            #     }
            # }
        }
    }
}

response = es_client.search(index = index_name , body = serch_query)
response['hits']

{'total': {'value': 865, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions-hw1',
   '_id': 'isHuM5ABryKeA6293vFj',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions-hw1',
   '_id': 'mMHuM5ABryKeA629q--g',
   '_score': 75.54128,
   '_source': {'text': 'In case running pgcli  locally causes issues or you do not want to install it locally you can use it running in a Docker container instead.\nBelow the usage with values used in the videos of the course for:\

In [36]:
#Q4
query = 'How do I execute a command in a running docker container?'
serch_query = {
    "size": 3,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^4", "text"],
                    "type": "best_fields"
                }
            }
            ,
            "filter": {
                "term": {
                    "course": "machine-learning-zoomcamp"
                }
            }
        }
    }
}

response = es_client.search(index = index_name , body = serch_query)
response['hits']


{'total': {'value': 345, 'relation': 'eq'},
 'max_score': 84.050095,
 'hits': [{'_index': 'course-questions-hw1',
   '_id': 'isHuM5ABryKeA6293vFj',
   '_score': 84.050095,
   '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
    'section': '5. Deploying Machine Learning Models',
    'question': 'How do I debug a docker container?',
    'course': 'machine-learning-zoomcamp'}},
  {'_index': 'course-questions-hw1',
   '_id': 'qcHuM5ABryKeA6294fGD',
   '_score': 51.04628,
   '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you 

In [37]:
#  Q5
context_template = """
Q: {question}
A: {text}
""".strip()



context = ""
    
for doc in response['hits']['hits']:   
    
    context = context + context_template.format(question=doc['_source']['question'], text=doc['_source']['text']).strip() +'\n\n'        

prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()

q = "How do I execute a command in a running docker container?"
prompt = prompt_template.format(question = q , context = context.strip())

print(len(prompt))

1462


In [43]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [47]:
#Q6
len(encoding.encode(prompt))

322

In [45]:
print(prompt)

You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: How do I execute a command in a running docker container?

CONTEXT:
Q: How do I debug a docker container?
A: Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.
docker run -it --entrypoint bash <image>
If the container is already running, execute a command in the specific container:
docker ps (find the container-id)
docker exec -it <container-id> bash
(Marcos MJD)

Q: How do I copy files from my local machine to docker container?
A: You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:
To copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:
docker cp /path/to/local/file_or_directory container_id:

In [48]:
from openai import OpenAI
client = OpenAI()

In [49]:
# Bonus: generating the answer (ungraded)
response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)

In [50]:
response

ChatCompletion(id='chatcmpl-9c4RJAbGnGkWdBSzZnnzvw5dhNhLv', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="To execute a command in a running Docker container, you can use the `docker exec` command. Follow these steps:\n\n1. First, identify the container ID of the running container by using the `docker ps` command:\n   ```\n   docker ps\n   ```\n\n2. Then, use the `docker exec` command to execute your desired command inside the container. Here's how to start a bash shell, for example:\n   ```\n   docker exec -it <container-id> bash\n   ```\n\nReplace `<container-id>` with the actual ID of the container you obtained from the `docker ps` command. This method allows you to execute any command in the running container.", role='assistant', function_call=None, tool_calls=None))], created=1718860557, model='gpt-4o-2024-05-13', object='chat.completion', system_fingerprint='fp_5e6c71d4a8', usage=CompletionUsage(completion_tokens=136, prompt_t

In [ ]:
# Bonus: calculating the costs (ungraded)
